In [14]:
!pip install pulp

In [18]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
from math import radians, cos, sin, sqrt, atan2

# 거리 계산 함수 (Haversine 공식)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0 # 지구의 반지름 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance


# 데이터 읽기
df = pd.read_excel('min_max_scaled_df.xlsx')

# 예제 데이터 (위 코드와 연결하여 수정해야 하는 부분)
# df는 '행정구역', '위도', '경도', '수요지수', '인프라지수', '접근성지수', '환경지수', '커버링 반경' 등의 열을 갖는다고 가정

# 커버링 반경 설정
cover_radius = 1.0  # 단위: km

# MCLP 설정
def solve_mclp(df):
    num_facilities = len(df)
    num_to_select = 5  # 5개의 그룹홈을 선택한다고 가정

    # 문제 설정
    model = LpProblem(name="MCLP", sense=LpMaximize)

    # 결정 변수
    x = [LpVariable(f'x_{i}', cat='Binary') for i in range(num_facilities)]

    # 목표 함수: 최대 수요 커버리지
    model += lpSum(df.loc[i, '수요 지수'] * x[i] for i in range(num_facilities))

    # 각 커버링 반경을 찾습니다
    for i in range(num_facilities):
        coverage_indices = []
        for j in range(num_facilities):
            # 모든 j에 대해서, i에서 j로의 거리를 계산
            distance = haversine(df.loc[i, '위도'], df.loc[i, '경도'], df.loc[j, '위도'], df.loc[j, '경도'])
            if distance <= cover_radius:
                coverage_indices.append(j)
        # 커버링 조건 추가
        model += lpSum(x[j] for j in coverage_indices) >= x[i]

    # 선택 제한
    model += lpSum(x[i] for i in range(num_facilities)) == num_to_select

    # 문제 해결
    model.solve()

    # 선택된 인덱스 출력
    selected_indices = [i for i in range(num_facilities) if x[i].value() == 1]

    return selected_indices

# MCLP 문제 풀기
optimal_locations = solve_mclp(df)

# 최적 위치 출력
print("선택된 그룹홈 위치:")
for idx in optimal_locations:
    print(df.loc[idx, ['행정구역', '위도', '경도']])

optimal_df = df.loc[optimal_locations, ['행정구역', '위도', '경도']]

# 결과를 Excel 파일로 저장
output_file_path = '달서구 MCLP.xlsx'
optimal_df.to_excel(output_file_path, index=False)



선택된 그룹홈 위치:
행정구역           용산동
위도       35.856591
경도      128.531147
Name: 6, dtype: object
행정구역           월성동
위도       35.819585
경도      128.522523
Name: 9, dtype: object
행정구역           진천동
위도        35.81273
경도      128.525665
Name: 10, dtype: object
행정구역           상인동
위도        35.81391
경도      128.539966
Name: 12, dtype: object
행정구역           송현동
위도       35.829043
경도      128.551403
Name: 14, dtype: object
